# DNN(working)

** - Generalize DNN structure  **

# 1. Load Data 

### 1.1 Import packages

**import pixiedust** is for debugging jupyter notebook.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys
import platform

from time import gmtime, strftime

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


from keras import optimizers
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical 
from keras.callbacks import EarlyStopping, ModelCheckpoint,Callback

import keras
print("keras Version:", keras.__version__ , "\nTensorflow Version:", tf.VERSION, "\nPython Version:", platform.python_version()) 
from keras.utils.data_utils import Sequence

import keras.backend as K

import matplotlib.pyplot as plt
#import pixiedust #jupyter notebook debugger

Using TensorFlow backend.


keras Version: 2.2.4 
Tensorflow Version: 1.12.0 
Python Version: 3.6.8


In [2]:
tf.enable_eager_execution()

### To control GPU usage

### Read parameters from .mat files
some read-only version names for ease use.

### 1.2 Constants and File Names

**Code_VERSION** is the version of this notebook. This is used for naming weights files

***Note: This will be different in each notebook.*** 

In [5]:
Code_VERSION = "_e19v41"
File_path = "/data/knayem"

Necessary constants/file names that are used through out the code.

In [10]:
PARAM_FILE = "./dnn_models/DNN_params.mat"

TrainData_FILE = File_path +"/dnn_models/Train_datas_spec.mat"
DevData_FILE = File_path+"/dnn_models/CrossValidation_datas_spec.mat"
TestData_FILE = File_path+"/dnn_models/Test_datas_spec.mat"

Following filenames are used to save weights, 

**MODEL_FILE: ** to save best model/weights after an epoch cycle (if weights need to be updated)

**SAVE_MODEL_FILE: ** to save final/best model/weights after a training.

**LOG_FILE: ** to print log


***Note: This will be different in each notebook.*** 

In [7]:
# Best model, after a training cycle
MODEL_FILE = File_path +"/dnn_models/dnn_weights"+ Code_VERSION+"_{epoch:02d}.h5"

# Final saved file
SAVE_MODEL_FILE = File_path +"/dnn_models/dnn_model_final"+ Code_VERSION+".h5"

LOG_FILE = File_path +"/dnn_models/dnn_log"+Code_VERSION+".txt"

Following filenames are used as final output, 

**OUTPUT_FILE: ** Output of test. After testing on all test files, we save the output spectogram for each testing file.

***Note: This will be different in each notebook.*** 

In [8]:
# estimated Spectogram for test dataset
OUTPUT_FILE = File_path +"/dnn_models/dnn_Spec"+_+".mat"

### 1.3 Class defination to read Matlab data

In [14]:
class Opts:
#     opts_dict = dict()

    def __init__(self, FILE_PARA, FILE_DATA="", FILE_TEST=""):
        
        # Basic parameters
        with h5py.File(FILE_PARA, 'r') as f:
            key_list = list(f.keys())
            print('Opt keys:')

            for e,(k, v) in enumerate(f['opts'].items()):

                print("{0}->{1},".format(e,k), end="")

                if k == 'ARMA_order':
                    self.ARMA_order = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.ARMA_order
                elif k == 'ada_grad_eps':
                    self.ada_grad_eps = np.array(v)[0][0]
#                     self.opts_dict[k] = self.ada_grad_eps
                elif k == 'ada_sgd_scale':
                    self.ada_sgd_scale = np.array(v)[0][0]
#                     self.opts_dict[k] = self.ada_sgd_scale
                elif k == 'amra_order':
                    self.amra_order = int(np.array(v)[0][0])
                elif k == 'change_momentum_point':
                    self.change_momentum_point = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.change_momentum_point
                elif k == 'clip_level':
                    self.clip_level = int(np.array(v)[0][0])
                elif k == 'cost_function':
                    self.cost_function = ""
                    for c in np.array(v):
                        self.cost_function += chr(c[0])

#                     self.opts_dict[k] = self.cost_function

                elif k == 'cv_interval':
                    self.cv_interval = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.cv_interval
                elif k == 'dim_input':
                    self.dim_input = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.dim_input
                    print("(",k,"=",self.dim_input,")",end=" ")
                elif k == 'dim_output':
                    self.dim_output = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.dim_output
                    print("(",k,"=",self.dim_output,")",end=" ")
                elif k == 'drop_ratio':
                    self.drop_ratio = np.array(v)[0][0]
#                     self.opts_dict[k] = self.drop_ratio
                elif k == 'eval_on_gpu':
                    self.eval_on_gpu = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.
                elif k == 'feawin':
                    self.feawin = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.feawin
                elif k == 'final_momentum':
                    self.final_momentum = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.final_momentum
                elif k == 'Fs':
                    self.Fs = int(np.array(v)[0][0])
                elif k == 'fRange':
                    self.fRange = int(np.array(v)[0][0])
                elif k == 'hid_struct':
                    self.hid_struct = np.array(v)
#                     self.opts_dict[k] = self.hid_struct
                elif k == 'hopsize':
                    self.hopsize = int(np.array(v)[0][0])
                elif k == 'initial_momentum':
                    self.initial_momentum = np.array(v)[0][0]
#                     self.opts_dict[k] = self.initial_momentum
                elif k == 'isDropout':
                    self.isDropout = 0
#                     self.opts_dict[k] = self.isDropout
                elif k == 'isDropoutInput':
                    self.isDropoutInput = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.isDropoutInput
                elif k == 'isGPU':
                    self.isGPU = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.isGPU
                elif k == 'isNormalize':
                    self.isNormalize = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.isNormalize
                elif k == 'isPretrain':
                    self.isPretrain = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.isPretrain
                elif k == 'labwin':
                    self.labwin = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.labwin
                elif k == 'labeltype':
                    self.labeltype = int(np.array(v)[0][0])
                elif k == 'labcompress':
                    self.labcompress = int(np.array(v)[0][0])
                elif k == 'learner':
                    self.learner = ""
                    for c in np.array(v):
                        self.learner += chr(c[0])
#                     self.opts_dict[k] = self.learner
                elif k == 'logistic_max':
                    self.logistic_max = int(np.array(v)[0][0])
                elif k == 'logistic_steep':
                    self.logistic_max = int(np.array(v)[0][0])
                elif k == 'net_struct':
                    self.net_struct = np.array(v)
#                     for n_s in np.array(v):
#                         print('Opts Net Stuct:',n_s[0])
#                     self.opts_dict[k] = self.net_struct
                elif k == 'noise':
                    self.noise = int(np.array(v)[0][0])
                elif k == 'nfft':
                    self.nfft = int(np.array(v)[0][0])
                elif k == 'numGammatoneChans':
                    self.numGammatoneChans = int(np.array(v)[0][0])
                elif k == 'overlap':
                    self.overlap = int(np.array(v)[0][0])
                elif k == 'overlap_len':
                    self.overlap_len = int(np.array(v)[0][0])
                elif k == 'rbm_batch_size':
                    self.rbm_batch_size = int(np.array(v)[0][0])
                    print("(self.rbm_batch_size=",self.rbm_batch_size,")",end=" ")
#                     self.opts_dict[k] = self.rbm_batch_size
                elif k == 'rbm_learn_rate_binary':
                    self.rbm_learn_rate_binary = np.array(v)
#                     self.opts_dict[k] = self.rbm_learn_rate_binary
                elif k == 'rbm_learn_rate_real':
                    self.rbm_learn_rate_real = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.rbm_learn_rate_real
                elif k == 'rbm_max_epoch':
                    self.rbm_max_epoch = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.rbm_max_epoch
                elif k == 'save_on_fly':
                    self.save_on_fly = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.save_on_fly
                elif k == 'sgd_batch_size':
                    self.sgd_batch_size = int(np.array(v)[0][0]) # BATCH_SIZE for training net
                    print("(self.sgd_batch_size:",self.sgd_batch_size,")",end=" ")
#                     self.opts_dict[k] = self.sgd_batch_size
                elif k == 'sgd_learn_rate':
                    self.sgd_learn_rate = np.array(v)
#                     self.opts_dict[k] = self.sgd_learn_rate
                elif k == 'sgd_max_epoch':
                    self.sgd_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.sgd_max_epoch
                elif k == 'split_tanh1_c1':
                    self.split_tanh1_c1 = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.split_tanh1_c1
                elif k == 'split_tanh1_c2':
                    self.split_tanh1_c2 = int(np.array(v)[0][0])
#                     self.opts_dict[k] = self.split_tanh1_c2
                elif k == 'unit_type_hidden':
                    self.unit_type_hidden = ""
                    for c in np.array(v):
                        self.unit_type_hidden += chr(c[0])

                elif k == 'unit_type_output':
                    self.unit_type_output = ""
                    for c in np.array(v):
                        self.unit_type_output += chr(c[0])
                elif k == 'winlen':
                    self.winlen = int(np.array(v)[0][0])
                elif k == 'feawin':
                    self.feawin = int(np.array(v)[0][0])

                        

    # Read different data files (Train, Dev, Test)
    def read_data(self, FILE_NAME, DATA_TYPE):
        print(FILE_NAME, os.path.isfile(FILE_NAME))

        with h5py.File(FILE_NAME, 'r') as f:
            # print('\n\nFile name <{0}>\nOpt h5py keys (Total {1}):'.format(FILE_TEST,len(f.keys())) )

            for k, v in f.items():
                if DATA_TYPE.lower() == 'train':
                    # Features (input)
                    if k == 'trData':
                        self.trData = np.transpose(np.array(v))
                    elif k == 'trLabel_r':
                        self.trLabel_r = np.transpose(np.array(v))
                    elif k == 'trLabel_i':
                        self.trLabel_i = np.transpose(np.array(v))
                    elif k == 'numframes':
                        self.trNumframes = np.transpose(np.array(v))
                    elif k == 'spec_clean_r':
                        self.trCleanSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_clean_i':
                        self.trCleanSpec_i = np.transpose(np.array(v))
                    elif k == 'spec_mixture_r':
                        self.trMixtureSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_mixture_i':
                        self.trMixtureSpec_i = np.transpose(np.array(v))
                    elif k == 'cleanFilename':
                        self.trCleanFilename = np.transpose(np.array(v))
                    elif k == 'trFilename':
                        self.trFilename = np.transpose(np.array(v))
                
                elif DATA_TYPE.lower() == 'dev':
                    # Features (input)
                    if k == 'cvData':
                        self.cvData = np.transpose(np.array(v))
                    elif k == 'cvLabel_r':
                        self.cvLabel_r = np.transpose(np.array(v))
                    elif k == 'cvLabel_i':
                        self.cvLabel_i = np.transpose(np.array(v))
                    elif k == 'numframes':
                        self.cvNumframes = np.transpose(np.array(v))
                    elif k == 'spec_clean_r':
                        self.cvCleanSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_clean_i':
                        self.cvCleanSpec_i = np.transpose(np.array(v))
                    elif k == 'spec_mixture_r':
                        self.cvMixtureSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_mixture_i':
                        self.cvMixtureSpec_i = np.transpose(np.array(v))
                    elif k == 'cleanFilename':
                        self.cvCleanFilename = np.transpose(np.array(v))
                    elif k == 'cvFilename':
                        self.cvFilename = np.transpose(np.array(v))
            
                elif DATA_TYPE.lower() == 'test':
                    # Features (input)
                    if k == 'teData':
                        self.teData = np.transpose(np.array(v))
                    elif k == 'teLabel_r':
                        self.teLabel_r = np.transpose(np.array(v))
                    elif k == 'teLabel_i':
                        self.teLabel_i = np.transpose(np.array(v))
                    elif k == 'numframes':
                        self.teNumframes = np.transpose(np.array(v))
                    elif k == 'spec_clean_r':
                        self.teCleanSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_clean_i':
                        self.teCleanSpec_i = np.transpose(np.array(v))
                    elif k == 'spec_mixture_r':
                        self.teMixtureSpec_r = np.transpose(np.array(v))
                    elif k == 'spec_mixture_i':
                        self.teMixtureSpec_i = np.transpose(np.array(v))
                    elif k == 'cleanFilename':
                        self.teCleanFilename = np.transpose(np.array(v))
                    elif k == 'teFilename':
                        for e,x in enumerate(np.array(v)):
                            print(e,x)
                        self.teFilename = np.transpose(np.array(v))
            
            # Display statistics
            if DATA_TYPE.lower() == 'train':
#                 self.trLabel = np.concatenate((self.trLabel_r, self.trLabel_i), axis=1)
                self.display_stat(DATA_TYPE="train")
                
            elif DATA_TYPE.lower() == 'dev':
#                 self.cvLabel = np.concatenate((self.cvLabel_r, self.cvLabel_i), axis=1)
                self.display_stat(DATA_TYPE="dev")
                
            elif DATA_TYPE.lower() == 'test':
#                 self.teLabel = np.concatenate((self.teLabel_r, self.teLabel_i), axis=1)
                self.display_stat(DATA_TYPE="test")
                
                
                
    # Display the simple statistics of Data (train, dev, test)
    def display_stat(self, DATA_TYPE):
        if DATA_TYPE.lower() == 'train':
            print("\nSummary->[TRAIN DATA]")
            print("trNumframes.shape=", self.trNumframes.shape)
            
#             shp, m, v, s, mn, mx = self.data_stat(self.trData)
#             print("trData.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.trLabel_r)
#             print("trLabel_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.trLabel_i)
#             print("trLabel_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.trLabel)
#             print("trLabel.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.trCleanSpec_r)
            print("trCleanSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.trCleanSpec_i)
            print("trCleanSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.trMixtureSpec_r)
            print("trMixtureSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.trMixtureSpec_i)
            print("trMixtureSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            print("trCleanFilename.shape={0}, trFilename.shape={0}".
                              format(self.trCleanFilename.shape, self.trFilename.shape) )
            
        elif DATA_TYPE.lower() == 'dev':
            print("\nSummary->[DEV DATA]")
            print("cvNumframes.shape=", self.cvNumframes.shape)
            
#             shp, m, v, s, mn, mx = self.data_stat(self.cvData)
#             print("cvData.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.cvLabel_r)
#             print("cvLabel_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.cvLabel_i)
#             print("cvLabel_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.cvLabel)
#             print("cvLabel.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.cvCleanSpec_r)
            print("cvCleanSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.cvCleanSpec_i)
            print("cvCleanSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.cvMixtureSpec_r)
            print("cvMixtureSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.cvMixtureSpec_i)
            print("cvMixtureSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            print("cvCleanFilename.shape={0}, cvFilename.shape={0}".
                              format(self.cvCleanFilename.shape, self.cvFilename.shape) )

            
        elif DATA_TYPE.lower() == 'test':
            print("\nSummary->[TEST DATA]")
            print("teNumframes.shape=", self.teNumframes.shape)
            
#             shp, m, v, s, mn, mx = self.data_stat(self.teData)
#             print("teData.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.teLabel_r)
#             print("teLabel_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.teLabel_i)
#             print("teLabel_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
#             shp, m, v, s, mn, mx = self.data_stat(self.teLabel)
#             print("teLabel.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
#                               format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.teCleanSpec_r)
            print("teCleanSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.teCleanSpec_i)
            print("teCleanSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.teMixtureSpec_r)
            print("teMixtureSpec_r.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            shp, m, v, s, mn, mx = self.data_stat(self.teMixtureSpec_i)
            print("teMixtureSpec_i.shape={0}, mean={1:.4f}, variance={2:.4f}, std={3:.4f}, range=[{4:.4f},{5:.4f}]".
                              format(shp, m, v, s, mn, mx) )
            print("teCleanFilename.shape={0}, trFilename.shape={0}".
                              format(self.teCleanFilename.shape, self.teFilename.shape) )
                        
    # Helper function for display, returns actual values
    def data_stat(self, data):
        return data.shape,np.mean(data),np.var(data),np.std(data),np.amin(data),np.amax(data)
    

Define an object which does the parameter (Matlab *Opts* object) reading.

In [15]:
opts = Opts(PARAM_FILE)

Opt keys:
0->ARMA_order,1->Fs,2->ada_grad_eps,3->ada_sgd_scale,4->arma_order,5->change_momentum_point,6->clip_level,7->cost_function,8->cv_interval,9->dim_input,( dim_input = 1230 ) 10->dim_output,( dim_output = 963 ) 11->drop_ratio,12->eval_on_gpu,13->fRange,14->feawin,15->final_momentum,16->hid_struct,17->hop_size,18->hopsize,19->initial_momentum,20->isDropout,21->isDropoutInput,22->isGPU,23->isNormalize,24->isPretrain,25->labcompress,26->labeltype,27->labwin,28->learner,29->logistic_max,30->logistic_steep,31->net_struct,32->nfft,33->noise,34->numGammatoneChans,35->overlap,36->overlap_len,37->rbm_batch_size,(self.rbm_batch_size= 1024 ) 38->rbm_learn_rate_binary,39->rbm_learn_rate_real,40->rbm_max_epoch,41->save_on_fly,42->sgd_batch_size,(self.sgd_batch_size: 1024 ) 43->sgd_learn_rate,44->sgd_max_epoch,45->split_tanh1_c1,46->split_tanh1_c2,47->tr_mu,48->tr_std,49->unit_type_hidden,50->unit_type_output,51->win_len,52->winlen,

### Read Train data 

In [13]:
opts.read_data(TrainData_FILE, DATA_TYPE="train")

/data/knayem/dnn_models/Train_datas_spec.mat True

Summary->[TRAIN DATA]
trNumframes.shape= (60000, 1)
trCleanSpec_r.shape=(7807680, 321), mean=0.0021, variance=0.2995, std=0.5473, range=[-46.6837,39.0942]
trCleanSpec_i.shape=(7807680, 321), mean=-0.0000, variance=0.2909, std=0.5394, range=[-50.2884,37.3594]
trMixtureSpec_r.shape=(7807680, 321), mean=0.0060, variance=3.2893, std=1.8136, range=[-285.0552,274.8278]
trMixtureSpec_i.shape=(7807680, 321), mean=-0.0000, variance=3.0763, std=1.7539, range=[-134.2137,120.0675]
trCleanFilename.shape=(6, 1), trFilename.shape=(6, 1)


### Read Dev (validation) data

In [10]:
opts.read_data(DevData_FILE, DATA_TYPE="dev")

./dnn_models/CrossValidation_datas.mat True

Summary->[DEV DATA]
cvNumframes.shape= (3300, 1)
cvData.shape=(449610, 1230), mean=-0.0000, variance=0.6076, std=0.7795, range=[-9.6298,10.4941]
cvLabel_r.shape=(449610, 963), mean=0.0286, variance=0.0213, std=0.1458, range=[-10.0000,10.0000]
cvLabel_i.shape=(449610, 963), mean=-0.0000, variance=0.0130, std=0.1138, range=[-10.0000,10.0000]
cvLabel.shape=(449610, 1926), mean=0.0143, variance=0.0173, std=0.1316, range=[-10.0000,10.0000]
cvCleanSpec_r.shape=(449610, 321), mean=0.0020, variance=0.3002, std=0.5479, range=[-40.8223,41.5895]
cvCleanSpec_i.shape=(449610, 321), mean=-0.0000, variance=0.2895, std=0.5381, range=[-39.2169,43.0322]
cvMixtureSpec_r.shape=(449610, 321), mean=0.0044, variance=3.2486, std=1.8024, range=[-121.7945,125.0182]
cvMixtureSpec_i.shape=(449610, 321), mean=-0.0001, variance=3.1470, std=1.7740, range=[-101.6771,93.7446]


### Read Test data

You can run this portion later, when you need this data. It will help with ram storage.

In [16]:
opts.read_data(TestData_FILE, DATA_TYPE="test")

/data/knayem/dnn_models/Test_datas_spec.mat True
0 [3707764736          2          1          1          1          1]

Summary->[TEST DATA]
teNumframes.shape= (21800, 1)
teCleanSpec_r.shape=(2897600, 321), mean=0.0020, variance=0.2996, std=0.5473, range=[-42.8964,38.0338]
teCleanSpec_i.shape=(2897600, 321), mean=-0.0000, variance=0.2918, std=0.5402, range=[-37.0904,45.6445]
teMixtureSpec_r.shape=(2897600, 321), mean=0.0057, variance=3.3611, std=1.8333, range=[-126.1488,136.7698]
teMixtureSpec_i.shape=(2897600, 321), mean=-0.0001, variance=3.1499, std=1.7748, range=[-129.6114,133.3650]


AttributeError: 'Opts' object has no attribute 'trCleanFilename'

# 2. Helper Functions

### 2.1 Next Batch Generator

#### b. DNN Batch Generator

Every time it returns a batch `(x, y)`. **x** is the features and **y** is the labels.


In [52]:
class DNN_DataGenerator_SPEC(keras.utils.Sequence):
    '''
    Generates Spectogram data for Keras
    '''
    
    def __init__(self, CYCLE, opts, batch_size=32, shuffle=True):
        '''
        Initialization
        '''
        self.opts = opts
        self.batch_size = batch_size
        self.CYCLE = CYCLE.lower()
        self.shuffle = shuffle
        
        self.BATCH_TYPE = 'floor' # {actual,floor}
        # 'actual' uses all frames, so last batch can be smaller than batch_size.
        # 'floor' may not use all the frames, but all the batches will be of equal size.
        
        self.__len__()    
        self.batchIDs = self.ready_batchID() 
        self.on_epoch_end()

        
    def __len__(self):
        '''
        Denotes the number of batches per epoch.

        returns:
            num_batch <int> = total number of batches
            
        variable:
            opts <object> = all the parametric data
            batch_size <int>        
            CYCLE <string>  = {train, dev, test}
            total_num_samples <int> = total number of frame, shape[0] of data
        '''

        if self.CYCLE.lower()=='train':
            self.total_num_samples = self.opts.trMixtureSpec_r.shape[0]

        elif self.CYCLE.lower()=='dev':
            self.total_num_samples = self.opts.cvMixtureSpec_r.shape[0]

        elif self.CYCLE.lower()=='test':
            self.total_num_samples = self.opts.teMixtureSpec_r.shape[0]
            self.BATCH_TYPE = 'actual'


        if self.BATCH_TYPE.lower() == 'actual':
            self.num_batch = math.ceil(self.total_num_samples/self.batch_size) 
            
        elif self.BATCH_TYPE.lower() == 'floor':
            self.num_batch = self.total_num_samples//self.batch_size 
            
        
        return int(self.num_batch)
    
    
    
    def ready_batchID(self):
        '''
        Create a 2D array with (start,end) frame number of each batch.

        returns:
            batchIDs <nparray.2D> = shape(number_of_batches, 2); 
                                    each row (s,e) is the inclusive start(s) and end(e) frame number
        variables:
            opts <object> = all the parametric data
            batch_size <int>        
            CYCLE <string>  = {train, dev, test} 
            total_num_samples <int> = total number of frame, shape[0] of data
        '''

        batchIDs = np.zeros((self.num_batch,2))
        # (s,e); s = the begining frame number and e = the ending frame number of a batch

        s = np.where( np.arange(self.total_num_samples)%self.batch_size == 0 )
        e = np.where( np.arange(self.total_num_samples)%self.batch_size == self.batch_size-1 )

        batchIDs[:, 0] = s[0][0:self.num_batch]

        # when all batches are equal of batch_size   
        if e[0].shape[0] >= self.num_batch:
            batchIDs[:, 1] = e[0][0:self.num_batch]

        # when last batch is smaller than batch_size
        elif e[0].shape[0] < self.num_batch:
            batchIDs[:self.num_batch-1, 1] = e[0][0:self.num_batch]
            batchIDs[-1,1] = self.total_num_samples-1
            

        return batchIDs

    

    def __getitem__(self, index):
        '''
        Generate one batch of data
        '''
        # Get the correcponding batch
        batch = self.batchIDs[index]
        
        # Find list of IDs
        list_IDs_temp = self.indexes[ int( batch[0]): int( batch[1])+1 ]
        
        # Generate data
        x, y = self.__data_generation(list_IDs_temp)

        return x, y

    
    def on_epoch_end(self):
        '''
        Updates indexes after each epoch
        '''
        self.indexes = np.arange(self.total_num_samples)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

            
    def __data_generation(self, list_IDs_temp):
        '''
        Generates data containing batch_size samples.
        '''
        if self.CYCLE.lower()=='train':
            x = ( self.opts.trMixtureSpec_r[ list_IDs_temp ]**2 + self.opts.trMixtureSpec_i[ list_IDs_temp ]**2 )**0.5
            y = ( self.opts.trCleanSpec_r[ list_IDs_temp ]**2 + self.opts.trCleanSpec_i[ list_IDs_temp ]**2 )**0.5

        elif self.CYCLE.lower()=='dev':
            x = ( self.opts.cvMixtureSpec_r[ list_IDs_temp ]**2 + self.opts.cvMixtureSpec_i[ list_IDs_temp ]**2 )**0.5
            y = ( self.opts.cvCleanSpec_r[ list_IDs_temp ]**2 + self.opts.cvCleanSpec_i[ list_IDs_temp ]**2 )**0.5

        elif self.CYCLE.lower()=='test':
            x = ( self.opts.teMixtureSpec_r[ list_IDs_temp ]**2 + self.opts.teMixtureSpec_i[ list_IDs_temp ]**2 )**0.5
            y = ( self.opts.teCleanSpec_r[ list_IDs_temp ]**2 + self.opts.teCleanSpec_i[ list_IDs_temp ]**2 )**0.5

        # do normalizeation
        return x, y
        

In [13]:
class DNN_DataGenerator_SPEC2(keras.utils.Sequence):
    '''
    Generates Spectogram data for Keras in sequence
    '''
    
    def __init__(self, CYCLE, opts, batch_size=32, shuffle=True):
        '''
        Initialization
        '''
        self.opts = opts
        self.batch_size = batch_size
        self.CYCLE = CYCLE.lower()
        self.shuffle = shuffle
        
        self.BATCH_TYPE = 'floor' # {actual,floor}
        # 'actual' uses all frames, so last batch can be smaller than batch_size.
        # 'floor' may not use all the frames, but all the batches will be of equal size.
        
        self.__len__()    
        self.batchIDs = self.ready_batchID() 
        self.on_epoch_end()

        
    def __len__(self):
        '''
        Denotes the number of batches per epoch.

        returns:
            num_batch <int> = total number of batches
            
        variable:
            opts <object> = all the parametric data
            batch_size <int>        
            CYCLE <string>  = {train, dev, test}
            total_num_samples <int> = total number of frame, shape[0] of data
        '''

        if self.CYCLE.lower()=='train':
            sampleFrames = self.opts.trNumframes
            sampleSource_r = self.opts.trMixtureSpec_r
            sampleSource_i = self.opts.trMixtureSpec_i
            
            cleanSource_r = self.opts.trCleanSpec_r
            cleanSource_i = self.opts.trCleanSpec_i
            
        elif self.CYCLE.lower()=='dev':
            sampleFrames = self.opts.cvNumframes
            sampleSource_r = self.opts.cvMixtureSpec_r
            sampleSource_i = self.opts.cvMixtureSpec_i
            
            cleanSource_r = self.opts.cvCleanSpec_r
            cleanSource_i = self.opts.cvCleanSpec_i
            
        elif self.CYCLE.lower()=='test':
            sampleFrames = self.opts.teNumframes
            sampleSource_r = self.opts.teMixtureSpec_r
            sampleSource_i = self.opts.teMixtureSpec_i
            
            cleanSource_r = self.opts.teCleanSpec_r
            cleanSource_i = self.opts.teCleanSpec_i

            
        sampleFrames = sampleFrames.flatten()
        cumulativeFrames = np.cumsum(sampleFrames)
        
        self.allSamples_r = None
        self.allSamples_i = None
        self.cleanSamples_r = None
        self.cleanSamples_i = None
        
        for indx in range(len(cumulativeFrames)):

            prev = sampleSource_r[indx-1] if indx !=0 else 0
            now = sampleSource_r[indx]
            self.allSamples_r = sampleSource_r[prev:now] if self.allSamples_r is None else np.concatenate( (self.allSamples_r, sampleSource_r[prev:now]), axis=1) 
            self.allSamples_i = sampleSource_i[prev:now] if self.allSamples_i is None else np.concatenate( (self.allSamples_i, sampleSource_i[prev:now]), axis=1) 
            
            self.cleanSamples_r = cleanSource_r[prev:now] if self.cleanSamples_r is None else np.concatenate( (self.allSamples_r, cleanSource_r[prev:now]), axis=1) 
            self.cleanSamples_i = cleanSource_i[prev:now] if self.cleanSamples_i is None else np.concatenate( (self.allSamples_i, cleanSource_i[prev:now]), axis=1) 
            
            n_emptyFrame = sampleFrames%self.batch_size
            if n_emptyFrame:
                self.allSamples_r = np.concatenate( (self.allSamples_r, np.zeros(n_emptyFrame,321)), axis=1)
                self.allSamples_i = np.concatenate( (self.allSamples_i, np.zeros(n_emptyFrame,321)), axis=1)
                
                self.cleanSamples_r = np.concatenate( (self.cleanSamples_r, np.zeros(n_emptyFrame,321)), axis=1)
                self.cleanSamples_i = np.concatenate( (self.cleanSamples_i, np.zeros(n_emptyFrame,321)), axis=1)
                
                
        self.total_num_samples = self.allSamples_r.shape[0]
        
        if self.BATCH_TYPE.lower() == 'actual':
            self.num_batch = math.ceil(self.total_num_samples/self.batch_size) 
        elif self.BATCH_TYPE.lower() == 'floor':
            self.num_batch = self.total_num_samples//self.batch_size


        return int(self.num_batch)
    
    
    
    def ready_batchID(self):
        '''
        Create a 2D array with (start,end) frame number of each batch.

        returns:
            batchIDs <nparray.2D> = shape(number_of_batches, 2); 
                                    each row (s,e) is the inclusive start(s) and end(e) frame number
        variables:
            opts <object> = all the parametric data
            batch_size <int>        
            CYCLE <string>  = {train, dev, test} 
            total_num_samples <int> = total number of frame, shape[0] of data
        '''

        batchIDs = np.zeros((self.num_batch,2))
        # (s,e); s = the begining frame number and e = the ending frame number of a batch

        s = np.where( np.arange(self.total_num_samples)%self.batch_size == 0 )
        e = np.where( np.arange(self.total_num_samples)%self.batch_size == self.batch_size-1 )

        batchIDs[:, 0] = s[0][0:self.num_batch]

        # when all batches are equal of batch_size   
        if e[0].shape[0] >= self.num_batch:
            batchIDs[:, 1] = e[0][0:self.num_batch]

        # when last batch is smaller than batch_size
        elif e[0].shape[0] < self.num_batch:
            batchIDs[:self.num_batch-1, 1] = e[0][0:self.num_batch]
            batchIDs[-1,1] = self.total_num_samples-1

        return batchIDs

    

    def __getitem__(self, index):
        '''
        Generate one batch of data
        '''
        # Get the correcponding batch
        batch = self.batchIDs[index]
        
        # Find list of IDs
        list_IDs_temp = self.indexes[ int( batch[0]): int( batch[1])+1 ]
        
        # Generate data
        x, y = self.__data_generation(list_IDs_temp)

        return x, y

    
    def on_epoch_end(self):
        '''
        Updates indexes after each epoch
        '''
        self.indexes = np.arange(self.total_num_samples)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

            
    def __data_generation(self, list_IDs_temp):
        '''
        Generates data containing batch_size samples.
        '''
        x = ( self.allSamples_r[ list_IDs_temp ]**2 + self.allSamples_i[ list_IDs_temp ]**2 )**0.5
        y = ( self.cleanSamples_r[ list_IDs_temp ]**2 + self.cleanSamples_i[ list_IDs_temp ]**2 )**0.5

        return x, y
        

### 2.2 Cost Functions

#### Unwrapping

Need to signal approximation to unwrap.

#### a. Mask Approximation

In [32]:
def lossFunction_SPEC(yTrue,yPred):
    
    r, c = K.shape(yTrue)[0], K.shape(yTrue)[1]
    
    samples = K.cast(r,dtype='float32') 
    # because by default, K.shape tensors are dtype int32

    cost_real = K.sum(K.square(yTrue[:, :c//2]- yPred[:, :c//2]))
    cost_imag = K.sum(K.square(yTrue[:, c//2:]- yPred[:, c//2:]))
    
    return 0.5*(cost_real+cost_imag)/samples

#### b. Signal Approximation

### 2.1 Model Parameters

In [33]:
#opts.sgd_batch_size
# batch_size = 5

epochs = 80
batch_size = 32

n_input = 321  # data input
n_hidden_1 = 321  # 1st layer number of neurons
n_hidden_2 = 321  # 2nd layer number of neurons
n_hidden_3 = 321  # 3rd layer number of neurons

**n_train**, **n_train_dim** = ```<number of samples for training, input dimentions>```, final will be (1951920, 1230)

**n_dev**, **n_dev_dim** = ```<number of samples for development, input dimentions>```, final will be (449610, 1230)

**n_test**, **n_test_dim** =```<number of samples for testing, input dimentions>```, final will be (72440, 1230)

In [34]:
n_train, n_train_dim = (195192, 321) # opts.trData.shape = (1951920, 1230)
n_dev, n_dev_dim = (44961, 321) # opts.cvData.shape =(449610, 1230)
n_test, n_test_dim = (72440, 321) # opts.teData.shape =(72440, 1230)

n_train_files = 15000 #15000
n_dev_files = 3300 # 3300
n_test_files = 545

n_classes = (321)

### Train

In [35]:
model = Sequential()

BatchNormalization()
model.add( Dense(n_hidden_1, activation='relu', input_shape=(n_input,)) )
BatchNormalization()
model.add( Dense(n_hidden_2, activation='relu') )
BatchNormalization()
model.add( Dense(n_hidden_3, activation='relu') )
BatchNormalization()
model.add(Dense(n_classes, activation='relu'))


# Set callback functions to early stop training and save the best model so far
# callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
#              ModelCheckpoint(filepath= MODEL_FILE, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = lossFunction_SPEC, optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

print(model.metrics_names)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 321)               103362    
_________________________________________________________________
dense_6 (Dense)              (None, 321)               103362    
_________________________________________________________________
dense_7 (Dense)              (None, 321)               103362    
_________________________________________________________________
dense_8 (Dense)              (None, 321)               103362    
Total params: 413,448
Trainable params: 413,448
Non-trainable params: 0
_________________________________________________________________
None
['loss', 'acc']


In [36]:
params = {'opts': opts,
          'batch_size': batch_size, 
          'shuffle': True}


# Generators
training_generator = DNN_DataGenerator_SPEC('Train', **params)
validation_generator = DNN_DataGenerator_SPEC('Dev', **params)

callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE, monitor='val_acc', save_best_only=True)]

In [37]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs = epochs,
                    use_multiprocessing=True,
                    workers=6, 
                    callbacks=callbacks)

model.save(SAVE_MODEL_FILE)

Epoch 1/80
60997/60997 [==============================] - 369s 6ms/step - loss: 11.5451 - acc: 0.5896 - val_loss: 11.1764 - val_acc: 0.6115
Epoch 1/80
Epoch 2/80
60997/60997 [==============================] - 333s 5ms/step - loss: 10.1950 - acc: 0.6044 - val_loss: 11.0825 - val_acc: 0.6206
Epoch 3/80
60997/60997 [==============================] - 359s 6ms/step - loss: 9.9682 - acc: 0.6079 - val_loss: 10.8754 - val_acc: 0.6288
Epoch 4/80
60997/60997 [==============================] - 338s 6ms/step - loss: 9.8513 - acc: 0.6101 - val_loss: 10.9853 - val_acc: 0.6214
Epoch 5/80
60997/60997 [==============================] - 335s 6ms/step - loss: 9.7740 - acc: 0.6112 - val_loss: 10.7263 - val_acc: 0.6245
Epoch 6/80
60997/60997 [==============================] - 336s 6ms/step - loss: 9.7315 - acc: 0.6124 - val_loss: 10.9792 - val_acc: 0.6302
Epoch 7/80
60997/60997 [==============================] - 330s 5ms/step - loss: 9.6976 - acc: 0.6130 - val_loss: 10.6784 - val_acc: 0.6248
Epoch 8/80
609

In [ ]:
from keras.layers import Layer

class FreqRecurrLayer(Dense):

    def __init__(self, units,
                 kernel_initializer='glorot_uniform',
                 activation=None,
                 use_bias=True,
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        
        self.output_dim = units
        super().__init__(units,
                 activation,
                 use_bias,
                 kernel_initializer,
                 bias_initializer,
                 kernel_regularizer,
                 bias_regularizer,
                 activity_regularizer,
                 kernel_constraint,
                 bias_constraint,
                 **kwargs)
        

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.input_batch = input_shape[-2]
        self.input_dim = input_shape[-1]

        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint,
                                      trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint,
                                        trainable=True)
        else:
            self.bias = None
            
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

        self.recurr = self.add_weight(name='recurr', 
                                      shape=(self.units,),
                                      initializer='uniform',
                                      trainable=True)
        
        super().build(input_shape)  # Be sure to call this at the end

        
    def call(self, inputs):
        output_temp = K.dot(inputs, self.kernel)
        if self.use_bias:
            output_temp = K.bias_add(output, self.bias, data_format='channels_last')
            
        for b in self.input_batch:
            
            
            
        if self.activation is not None:
            output = self.activation(output)
        return output
    
    
        res = K.dot(x, self.kernel)
        prev = 0
        temp_w = None
        for i in range(self.input_shape1):
            if i == 0:
                temp_w = res[0]
            else:
                temp_w = K.concatenate([temp_w, K.dot(res[i],self.recurr[i-1]) ], axis=0)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

### Load pretrained Mag weights

### Modify and retrain

### RNN 1
single bidirectional GRU layer

real+img (963+963)=1926-d output vector

### Testing

In [38]:
BEST_MODEL_FILE = "./dnn_models/dnn_weights"+ Code_VERSION+"_09.h5"

In [39]:
model = load_model(BEST_MODEL_FILE, custom_objects={'lossFunction_SPEC':lossFunction_SPEC})

In [53]:
params = {'opts': opts,
          'batch_size': batch_size, 
          'shuffle': False}

testing_generator = DNN_DataGenerator_SPEC('Test', **params)
y_hat = model.predict_generator(testing_generator, verbose=1)
print('y_hat shape', y_hat.shape)
sio.savemat(OUTPUT_FILE, {'y_hat':y_hat})

2264/2264 [==============================] - 3s 1ms/step
y_hat shape (72440, 321)
